In [42]:
import os
import shutil
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
import pydicom
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle as pkl
def dcm2nii(path_read, path_save):
    '''
    ## Convert Dicom Series Files to a Single NII File
    ### Args:
        path_read: The file folder containing dicom series files(No other files exits)
        path_save: The path you save the .nii/.nii.gz data file
    '''
    # GetGDCMSeriesIDs读取序列号相同的dcm文件
    series_id = sitk.ImageSeriesReader.GetGDCMSeriesIDs(path_read)
    # GetGDCMSeriesFileNames读取序列号相同dcm文件的路径，series[0]代表第一个序列号对应的文件
    series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(path_read, series_id[0])
    series_reader = sitk.ImageSeriesReader()
    series_reader.SetFileNames(series_file_names)
    image3d = series_reader.Execute()
    sitk.WriteImage(image3d, path_save)
def position(dicom_dir):
    # Get single dcm's physical position
    file = pydicom.filereader.dcmread(dicom_dir)
    cosines = file[0x20, 0x37].value
    ipp = file[0x20, 0x32].value
    a1 = np.zeros((3,))
    a2 = np.zeros((3,))
    for i in range(3):
        a1[i] = cosines[i]
        a2[i] = cosines[i+3]
    index = np.abs(np.cross(a1, a2))
    return ipp[np.argmax(index)]
# IDs=[]
Disea_map={'12154639':'DCM',
            '11909618':'DCM',
            '12145281':'LVNC',
            '12155770':'LVNC',
            '12138079':'心肌淀粉样变',
            '12146409':'心肌淀粉样变',
            '12149908':'埃布斯坦畸形',
            '12100298':'冠状动脉心肌梗死',
            '12157231':'冠状动脉心肌梗死',
            '11783530':'Myocarditis',
            '11892346':'RCM',
            '12150541':'RCM',
            '12149449':'RCM',
            '12117284':'RCM',
            '12136945':'肺动脉高压',
            '12131285':'肺动脉高压',
            '12123896':'类别10',
            '12016206':'类别10'}
def rename_dicom(dataroot='./DICOMs'):
    for root,dirs,files in os.walk(dataroot):
        # if len(dirs)==0:
        if True:
            for i in range(len(files)):
                if files[i]=='DICOMDIR.dcm':
                    os.remove(os.path.join(root,files[i]))
                    print(f'{os.path.join(root,files[i])} deleted')
                if not files[i].endswith('.dcm'):
                    os.rename(os.path.join(root,files[i]),os.path.join(root,f'{files[i]}.dcm'))
    return 0
def Rec_crop2D(mask,index):
    x_l,x_r,y_d,y_u=0,0,0,0
    for i in range(len(mask[index])):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_u=i-1
            break
    for i in range(len(mask[index])-1,0,-1):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_d=i+1
            break
    mask_t=np.transpose(mask[index])
    for i in range(len(mask_t)-1,0,-1):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_r=i+1
            break
    for i in range(len(mask_t)):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_l=i-1
            break
    # print(x_l,x_r,y_u,y_d)
    croped_mask=mask[index][y_u:y_d,x_l:x_r]
    return croped_mask,[x_l,x_r,y_u,y_d]
def crop(mask):
    x_l,x_r,y_d,y_u=0,0,0,0
    for i in range(len(mask)):
        if np.count_nonzero(mask[i]>0)>0:
            index=i
    for i in range(len(mask[index])):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_u=i-1
            break
    for i in range(len(mask[index])-1,0,-1):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_d=i+1
            break
    mask_t=np.transpose(mask[index])
    for i in range(len(mask_t)-1,0,-1):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_r=i+1
            break
    for i in range(len(mask_t)):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_l=i-1
            break
    # print(x_l,x_r,y_u,y_d)
    # croped_mask=mask[index][y_u:y_d,x_l:x_r]
    return x_l,x_r,y_u,y_d


def sort_dict(dictionary):
    sorted_keys = sorted(dictionary)
    new_map = {}
    for i in range(len(sorted_keys)):
        new_map[sorted_keys[i]] = dictionary[sorted_keys[i]]
    return new_map

def del_empty(root):
    for roots, dirs, files in os.walk(root):
        if len(dirs) == 0:
            if len(files) == 0:
                shutil.rmtree(roots)
                print(f'{roots}')
def mac_bug(root):
    for roots, dirs, files in os.walk(root):
        for i in range(len(files)):
            if files[i].startswith('._'):
                os.remove(os.path.join(roots, files[i]))
def norm_img(img):
    return np.uint8(255.0*(img-np.min(img))/(np.max(img)-np.min(img)))
def exclude_seg_files(file_list=[],segmentations=['Segmentation.nii','Segmentation.nii.gz','.DS_Store']):
    for seg_file in segmentations:
        if seg_file in file_list:
            file_list.remove(seg_file)
    return file_list
pos_map={0:'up',1:'mid',2:'down'}
def ds_store(root):
    for roots, dirs, files in os.walk(root):
        for i in range(len(files)):
            if '.DS_Store' == files[i]:
                os.remove(os.path.join(roots, files[i]))
                print(os.path.join(root, files[i]))
# rename_dicom(dataroot='/Users/airskcer/Downloads/HCM_20230113/')

0

In [1]:
def plot_mid_up_down(root,output):
    try:
        os.makedirs(output)
    except:
        pass
    for roots,dirs,files in os.walk(root):
        if len(dirs)==0 and 'SAX_data' in roots:
            pat_name=roots.split('/')[-1]
            if 'slice_mid.nii.gz' in files and 'slice_up.nii.gz' in files and 'slice_down.nii.gz' in files:
                mid=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,'slice_mid.nii.gz')))
                up=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,'slice_up.nii.gz')))
                down=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,'slice_down.nii.gz')))
                plt.figure(figsize=(15,15))
                plt.subplot(131)
                plt.imshow(up[0],cmap='gray')
                plt.subplot(132)
                plt.imshow(mid[0],cmap='gray')
                plt.subplot(133)
                plt.imshow(down[0],cmap='gray')
                plt.savefig(os.path.join(output,f'{pat_name}.jpg'))
# plot_mid_up_down(
#     root='/data/Airscker/VST_fusion_dataset_M1/Dataset/Abnormal_nifti_spacing_1.826', output='/data/Airscker/VST_fusion_dataset_M1/Dataset/up_mid_down_check')

In [3]:
def resample_volume(Origin='NII.nii.gz', volume=None, interpolator = sitk.sitkLinear, new_spacing = [1.7708333730698,1.7708333730698,1],output='Resampled.nii'):
    '''
    ## Resample MRI files to specified spacing, here we define spacing=[1.7708333730698,1.7708333730698,1]
    ### Args:
        Origin: The Original MRI file, MUST NOT BE DICOM!!!!!
        interpolater: The method of resampling
        new_spacing: The spacing we want to set
        output: The Output path of resampled MRI data
    ### Return:
        resample_image: The resampled MRI data
    '''
    if volume is None:
        volume = sitk.ReadImage(Origin)
    original_spacing = volume.GetSpacing()
    original_size = volume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, new_spacing)]
    resampled_image = sitk.Resample(volume,
                                    new_size,
                                    sitk.Transform(), 
                                    interpolator,
                                    volume.GetOrigin(), 
                                    new_spacing, volume.GetDirection(), 
                                    0,
                                    volume.GetPixelID())
    if output!='':
        sitk.WriteImage(resampled_image,output)
    return resampled_image

## Reslice FPS of SAXLGE, and delete the original discrete nifti files(optional)

In [36]:
def reslice_lge(root,fps=9):
    for roots,dirs,files in os.walk(root):
        if len(dirs)==0:
            files=exclude_seg_files(files)
            data_map={}
            for i in range(len(files)):
                data=sitk.ReadImage(os.path.join(roots,files[i]))
                data_map[data.GetOrigin()[-1]]=data
            data_map=sort_dict(data_map)
            new_array=[]
            for key in data_map.keys():
                array=sitk.GetArrayFromImage(data_map[key])
                if array.shape[0]>1:
                    array=np.array([array[array.shape[0]//2]])
                new_array.append(array)
            new_array=np.array(new_array).squeeze()
            # print(new_array.shape)
            origin=data_map[key].GetOrigin()
            spacing=data_map[key].GetSpacing()
            new_spacing=list(spacing)
            new_spacing[2]=spacing[2]*len(files)/float(fps)
            direction=data_map[key].GetDirection()
            pixelid=data_map[key].GetPixelID()
            size=data_map[key].GetSize()
            new_data=sitk.GetImageFromArray(new_array)
            new_data.SetDirection(direction)
            new_data.SetOrigin(origin)
            new_data.SetSpacing(spacing)
            new_size = [new_array.shape[1],new_array.shape[2],fps]
            resampled_image = sitk.Resample(new_data,new_size,sitk.Transform(),sitk.sitkLinear,origin,new_spacing,direction,0,new_data.GetPixelID())
            return resampled_image
def batch_reslice(root,fps=9,delete=True,mod='SAX_LGE_data'):
    error=[]
    available_data = []
    for roots,dirs,files in os.walk(root):
        if len(dirs)==0 and mod in roots:
            available_data.append(roots)
    bar=tqdm(range(len(available_data)))
    for i in bar:
        roots=available_data[i]
        original_files=os.listdir(roots)
        pat_name=roots.split('/')[-1]
        if os.path.exists(os.path.join(roots, f'{pat_name}_fps_{fps}.nii.gz')):
            continue
        try:
            new_data = reslice_lge(root=roots, fps=fps)
        except:
            print(f'Reslice failure with {roots}')
            error.append(roots)
            continue
        if sitk.GetArrayFromImage(new_data).shape[0]!=fps:
            print(f'ERROR occured with {roots}: {fps}/{sitk.GetArrayFromImage(new_data).shape[0]}')
            error.append(roots)
        else:
            if not os.path.exists(roots):
                os.makedirs(roots)
            sitk.WriteImage(new_data, os.path.join(
                roots, f'{pat_name}_fps_{fps}.nii.gz'))
            if delete:
                # shutil.rmtree(roots)
                for file in original_files:
                    os.remove(os.path.join(roots,file))
    return error
# new_data=reslice_lge(root='/Users/airskcer/Downloads/HCM_nifti-20221226/SAX_LGE_data/11828017_ZHANG_HONG_DA/')
# print(sitk.GetArrayFromImage(new_data).shape)
# sitk.WriteImage(new_data,'test.nii')
# del_empty('/Users/airskcer/Downloads/')s
# error=batch_reslice(root='/Users/airskcer/Downloads/SAX_LGE_data/')

## Map resolution from nifti to nifti

In [29]:
def resample_dataset2(spacing=[0.994,1.826],datapath='E:/VST_fusion/dataset'):
    error=[]
    for space in spacing:
        for roots,dirs,files in os.walk(datapath):
            if len(dirs)==0:
                files=exclude_seg_files(os.listdir(roots))
                for i in range(len(files)):
                    data_path=os.path.join(roots,files[i])
                    pre_data=sitk.ReadImage(data_path)
                    data_spacing=list(pre_data.GetSpacing())
                    data_spacing[0]=space
                    data_spacing[1]=space
                    new_data=resample_volume(Origin=data_path,new_spacing=data_spacing,output='')
                    if len(sitk.GetArrayFromImage(new_data))!=25 and 'LGE' not in data_path:
                        print(f'frame resample error: {data_path} {sitk.GetArrayFromImage(new_data).shape}')
                        error.append(data_path)
                    else:
                        save_path=roots.replace('nifti',f'nifti_spacing_{space}')
                        save_path=save_path.replace('dataset',f'dataset_spacing_{space}')
                        # save_path=save_path.replace('E:','F:')
                        if not os.path.exists(save_path):
                            os.makedirs(save_path)
                        save_path=os.path.join(save_path,files[i])
                        sitk.WriteImage(new_data,save_path)
                        print(f'{save_path} saved')
    return error
error=resample_dataset2(datapath='/Users/airskcer/Downloads/NP_nifti-20230109/',spacing=[0.994,1.826])
# error=resample_dataset2(datapath='/Users/airskcer/Downloads/RCM_nifti-20230107/',spacing=[0.994,1.826])

/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_13.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_mid.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_down.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_11.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_15.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_17.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_up.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11183087_LI_CHANG_AN/slice_18.nii.gz saved
/Users/airskcer/Downloads/NP_nifti_spacing_0.994-20230109/SAX_data/11028695_DAI_XIN_HUA/slice_mid.nii.gz saved
/Users

In [14]:
def check_spacings(root='/data/Airscker/VST_fusion_dataset_M1/Dataset/Abnormal_nifti_spacing_1.826'):
    pats = []
    for roots, dirs, files in os.walk(root):
        if len(dirs) == 0:
            pats.append(roots)
    bar = tqdm(range(len(pats)))
    error = []
    for i in bar:
        target = pats[i].replace('1.826', '0.994')
        if not os.path.exists(target):
            error.append(pats[i])
        else:
            pat_num = len(os.listdir(pats[i]))
            target_num = len(os.listdir(target))
            if pat_num != target_num:
                error.append(f'{pats[i]} {pat_num} {target_num}')
    print(len(error))
    for i in range(len(error)):
        print(error[i])
check_spacings(root='/Users/airskcer/Downloads/HCM_nifti_spacing_1.826-10s/')

100%|██████████| 40/40 [00:00<00:00, 17527.39it/s]

0


## check slices' data size, if every patient's slices' size isn't uniformed, error occurs

In [10]:
def check_size(dataroot='/Users/airskcer/Downloads/DCM_nifti-20230107/'):
    error={}
    folders=[]
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            folders.append(roots)
    bar=tqdm(range(len(folders)))
    for i in bar:
        files=exclude_seg_files(os.listdir(folders[i]))
        if len(files)==1:
            continue
        sizes=[]
        for j in range(len(files)):
            data=sitk.ReadImage(os.path.join(folders[i],files[j]))
            sizes.append(sitk.GetArrayFromImage(data).shape)
        sizes=np.array(sizes)
        if np.sum(np.std(sizes,axis=0))!=0:
            error[folders[i]]=sizes
    for key in error:
        print(key,error[key])
    return error
error=check_size(dataroot='/Users/airskcer/Downloads/LVNC_nifti-20221224/')

100%|██████████| 1316/1316 [02:04<00:00, 10.57it/s]


In [ ]:
def error_process(error_map: dict):
    """
    The error_process function is used to find the files that have different shapes and move them into a new folder.
    
    
    :param error_map: dict: Store the error information
    :return: The file names and the shape of the files that have an error
    :doc-author: Trelent
    """
    for path in error_map.keys():
        map={}
        files=exclude_seg_files(os.listdir(path))
        for i in range(len(files)):
            map[files[i]]=np.array(list(sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,files[i]))).shape))
        shapes=np.array(list(map.values()))
        new_shape=np.unique(shapes,axis=0)
        error_shape=None
        if len(files)>2:
            for i in range(len(new_shape)):
                count=0
                for j in range(len(shapes)):
                    if np.all(shapes[j]==new_shape[i]):
                        count+=1
                if count==1:
                    error_shape=new_shape[i]
        else:
            continue
        if error_shape is not None:
            for key in map:
                if np.all(error_shape==map[key]):
                    print(path,key,error_shape)
                    # if 'SAX_LGE_data' in path:
                    #     target = path.replace('SAX_LGE_data', '4CH_LGE_data')
                        # try:
                        #     os.makedirs(target)
                        # except:
                        #     pass
                        # try:
                        #     shutil.move(os.path.join(path,key),target)
                        # except:
                        #     print(f'{target} already exists')
                    # os.remove(os.path.join(path,key))
error_process(error_map=error)


/Users/airskcer/Downloads/LVNC_nifti-20221224/SAX_data/11143330_XU_DONG_XI slice_8.nii.gz [ 25 162 192]
/Users/airskcer/Downloads/LVNC_nifti-20221224/SAX_data/11470714_LIU_LI slice_8.nii.gz [  5 192 192]
/Users/airskcer/Downloads/LVNC_nifti-20221224/SAX_data/11555140_LIU_FANG_FANG slice_8.nii.gz [ 25 162 192]
/Users/airskcer/Downloads/LVNC_nifti-20221224/SAX_data/11850433_ZHANG_XIN slice_1.nii.gz [ 25 352 352]
/Users/airskcer/Downloads/LVNC_nifti-20221224/SAX_data/11313413_QIAN_XIAO_LIN slice_12.nii.gz [ 25 168 192]
/Users/airskcer/Downloads/LVNC_nifti-20221224/4CH_data/12108320_CHEN_HAO 12108320_CHEN_HAO_4.nii.gz [ 35 164 208]
/Users/airskcer/Downloads/LVNC_nifti-20221224/4CH_data/11133538_QIAN_XIAO_LIN 11133538_QIAN_XIAO_LIN_8.nii.gz [ 25 162 192]
/Users/airskcer/Downloads/LVNC_nifti-20221224/SAX_LGE_data/12108320_CHEN_HAO 12108320_CHEN_HAO_2.nii.gz [  5 256 256]
/Users/airskcer/Downloads/LVNC_nifti-20221224/4CH_LGE_data/12108320_CHEN_HAO 12108320_CHEN_HAO_3.nii.gz [  1 256 256]
/Use

## Statistic-slices of nifti data

In [22]:
def slice_stats(root_path):
    mods=['4CH_data','SAX_data','4CH_LGE_data','SAX_LGE_data']
    pats=[]
    info=[]
    for i in range(len(mods)):
        try:
            pats_mod=os.listdir(os.path.join(root_path,mods[i]))
            for j in range(len(pats_mod)):
                if pats_mod[j] not in pats:
                    pats.append(pats_mod[j])
        except:
            pass
    # print(root_path,len(pats))
    pats=exclude_seg_files(pats)
    for i in range(len(pats)):
        pat_info={'Class':root_path.split('/')[-1],'Name':pats[i]}
        for j in range(len(mods)):
            try:
                pat_folder=os.path.join(root_path,mods[j],pats[i])
                pat_files=exclude_seg_files(os.listdir(pat_folder))
                pat_info[mods[j]]=len(pat_files)
                if '4CH_LGE' in mods[j] and len(pat_files)>1:
                    print(pat_folder,len(pat_files))
            except:
                pass
        info.append(pat_info)
    return info
# roots=os.listdir('E:/nifti_original_space/')
# all_info=[]
# for i in range(len(roots)):
#     all_info+=slice_stats(os.path.join('E:/nifti_original_space/',roots[i]))
all_info=slice_stats('/Users/airskcer/Downloads/HCM_nifti-20230112/')

/Users/airskcer/Downloads/HCM_nifti-20230112/4CH_LGE_data/12007733_ZHOU_YONG 3
/Users/airskcer/Downloads/HCM_nifti-20230112/4CH_LGE_data/11630728_HOU_CHEN 3
/Users/airskcer/Downloads/HCM_nifti-20230112/4CH_LGE_data/12097381_WU_YU_MIN 2


In [23]:
import pandas as pd
print(len(all_info))
chart=pd.DataFrame(all_info,columns=['Class','Name','4CH_data','SAX_data','4CH_LGE_data','SAX_LGE_data'])
chart.to_csv('./fuwai_dataset/HCM_20230112//slice_info.csv',index=False)
chart.head(10)

679


,Class,Name,4CH_data,SAX_data,4CH_LGE_data,SAX_LGE_data
0,,11445696_HAO_YAN_BIN,1.0,8.0,NaN,NaN
1,,11846373_LU_WEI_WEI,2.0,8.0,1.0,8.0
2,,11938778_ZHANG_XIAO_LONG,2.0,8.0,NaN,9.0
3,,12119238_QIN_YU_MEI,2.0,9.0,1.0,9.0
4,,11105981_ZHAO_SHAN_MEI,1.0,8.0,1.0,8.0
5,,11865908_YU_FENG_YING,2.0,8.0,1.0,8.0
6,,11895149_YANG_XIU_YUN,2.0,8.0,1.0,8.0
7,,12118912_LI_JIE,1.0,9.0,1.0,9.0
8,,11942306_LI_JIA_YUAN,1.0,8.0,1.0,8.0
9,,11244559_LIU_TIE_WEN,1.0,8.0,1.0,8.0


## delete VTOL according to slice ID

In [ ]:
# path='/Users/airskcer/Downloads/HCM_nifti-20230102/4CH_data/'
# for roots,dirs,files in os.walk(path):
#     if len(dirs)==0:
#         pat_name=roots.split('/')[-1]
#         ids=[]
#         files=exclude_seg_files(files)
#         for i in range(len(files)):
#             ids.append(int(files[i].split('.nii.gz')[0].split('_')[-1]))
#         os.remove(os.path.join(roots,f'{pat_name}_{max(ids)}.nii.gz'))

## Save nifti as jpeg to check data

In [4]:
def check_data(root='/Users/airskcer/Downloads/HCM_nifti-20230102/',mod='4CH_data',output='/Users/airskcer/Downloads/check'):
    if not os.path.exists(output):
        os.makedirs(output)
    for roots,dirs,files in os.walk(root):
        if len(dirs)==0 and mod in roots:
            files=exclude_seg_files(files)
            # if len(files)<2:
            #     continue
            for i in range(len(files)):
                data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,files[i])))
                cv2.imwrite(os.path.join(output,files[i].replace('.nii.gz','.jpg')),norm_img(data[0]))
# check_data()

In [5]:
def del_data(root='/Users/airskcer/Downloads/HCM_nifti-20221226/', mod='4CH_data', output='/Users/airskcer/Downloads/check'):
    pats=os.listdir(output)
    for i in range(len(pats)):
        pats[i]=pats[i].split('.')[0]
    for roots, dirs, files in os.walk(root):
        if len(dirs) == 0 and mod in roots:
            files = exclude_seg_files(files)
            if len(files)==1:
                continue
            for i in range(len(files)):
                if files[i].split('.')[0] not in pats:
                    os.remove(os.path.join(roots,files[i]))
                    print(os.path.join(roots,files[i]))
# del_data()
# del_empty(root='/Users/airskcer/Downloads/NP_nifti-20221230/')

## Rename SAX_CINE/SAX_LGE slices with up/down/mid

In [25]:
def rename_mid(root,show=True):
    for roots, dirs, files in os.walk(root):
        if len(dirs) != 0:
            print(roots)
        if len(dirs) == 0 and 'SAX_data' in roots or 'SAX_LGE' in roots:
            files = exclude_seg_files(files)
            if len(files) >= 5 and ('slice_up.nii.gz' not in files or 'slice_mid.nii.gz' not in files or 'slice_down.nii.gz' not in files):
                slice_map = {}
                for i in range(len(files)):
                    data = sitk.ReadImage(os.path.join(roots, files[i]))
                    slice_map[data.GetOrigin()[-1]] = files[i]
                slice_pos = sorted(list(slice_map.keys()))
                # for i in range(len(slice_pos)):
                #     os.rename(os.path.join(roots,slice_map[slice_pos[i]]),os.path.join(roots,f'slice_{i}.nii.gz'))
                # os.rename(os.path.join(roots,f'slice_{len(slice_pos)//2}.nii.gz'), os.path.join(roots, 'slice_mid.nii.gz'))
                # os.rename(os.path.join(roots,f'slice_{len(slice_pos)//2+2}.nii.gz'), os.path.join(roots, 'slice_down.nii.gz'))
                # os.rename(os.path.join(roots,f'slice_{len(slice_pos)//2-2}.nii.gz'), os.path.join(roots, 'slice_up.nii.gz'))
                # if show:
                #     print(f'{roots} processed')
                os.rename(os.path.join(roots, slice_map[slice_pos[len(
                    slice_pos)//2]]), os.path.join(roots, 'slice_mid.nii.gz'))
                os.rename(os.path.join(roots, slice_map[slice_pos[len(
                    slice_pos)//2-2]]), os.path.join(roots, 'slice_down.nii.gz'))
                os.rename(os.path.join(roots, slice_map[slice_pos[len(
                    slice_pos)//2+2]]), os.path.join(roots, 'slice_up.nii.gz'))
            elif len(files)<5:
                print(roots, len(files))

rename_mid(root='/Users/airskcer/Downloads/NP_nifti-20230109/')
# rename_mid(root='/Users/airskcer/Downloads/RCM_nifti-20230107/')

/Users/airskcer/Downloads/NP_nifti-20230109/
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data
/Users/airskcer/Downloads/NP_nifti-20230109/4CH_data


## Resample Every Patient's Data

In [3]:
import os
import pydicom
import shutil
import numpy as np
def resample_dataset(dataroot='./DICOMs',output='./New_dicom/'):
    """Resample the dataset to a new directory .

    Args:
        dataroot (str, optional):The root path of all patients' dicom files. Defaults to './DICOMs'.
        output (str, optional):The output path of resampled dicom dataset. Defaults to './New_dicom/'.
    """
    error=[]
    count=0
    for paths,dirs,files in os.walk(dataroot):
        # if len(dirs)==0:
        if True:
            for i in range(len(files)):
                try:
                    info=pydicom.dcmread(os.path.join(paths,files[i]))
                    id=info['0010','0020'].value
                    name=str(info['0010','0010'].value).replace(' ','_')
                    mod=str(info['0008','103e'].value).replace(':','_')
                    date=str(info['0008','0020'].value)
                    ui=str(info['0008','0018'].value)
                except:
                    print(f'Basic info missed: {os.path.join(paths,files[i])}')
                    continue
                if name!='':
                    savepath=os.path.join(output,f'{id}_{name}',date,mod).replace('*','_').replace('?','_').replace('>','_').replace('<','_')
                    if not os.path.exists(savepath):
                        os.makedirs(savepath)
                    count+=1
                    try:
                        shutil.copyfile(os.path.join(paths,files[i]),os.path.join(savepath,f'{ui}.dcm'))
                    except:
                        print(f"Error occured when copying {os.path.join(paths,files[i])} to {os.path.join(savepath,f'{ui}.dcm')}")
                    # os.remove(os.path.join(paths,files[i]))
                else:
                    error.append(os.path.join(paths,files[i]))
        # for i in range(len(error)):
        #     print(error[i])
    return error
resample_dataset(dataroot='/Users/airskcer/Downloads/HCM_20230112/',output='/Users/airskcer/Downloads/HCM_Resampled_dicom-20230112')
# resample_dataset(dataroot='/Users/airskcer/Downloads/RCM_20230107/',output='/Users/airskcer/Downloads/RCM_Resampled_dicom-20230107')
# resample_dataset(dataroot='E:/BaiduNetdiskDownload/HHD/',output='F:/resampled_dicom/HHD_Resampled_dicom-20221217')

Basic info missed: /Users/airskcer/Downloads/HCM_20230112/.DS_Store
Basic info missed: /Users/airskcer/Downloads/HCM_20230112/.DS_Store.dcm
Basic info missed: /Users/airskcer/Downloads/HCM_20230112/20230108-HCM(425例图+425例信息) 王家鑫/.DS_Store.dcm
Basic info missed: /Users/airskcer/Downloads/HCM_20230112/20230108-HCM(425例图+425例信息) 王家鑫/2018.4th n=146/FU_ZHI_QIANG_046Y/Win10 (C) - ¿ì½Ý·½Ê½.lnk.dcm
Basic info missed: /Users/airskcer/Downloads/HCM_20230112/HCM yk2/.DS_Store.dcm
Basic info missed: /Users/airskcer/Downloads/HCM_20230112/HCM yk1/.DS_Store.dcm


[]

## Fix Series Number Error of Dicom Files, must be after data resample

In [6]:
def fix_seriesnum(root='./New_dicom/',threshold=100,min_fps=25,expand_mod=False):
    """Fixes series number according to the positions of dicom files

    Args:
        root (str, optional): The root path of resampled dataset. Defaults to './New_dicom/'.
        threshold (int, optional): If number of files oversize the threshold, this folder will be checked whether if there exists error of dicom SNs. Defaults to 100.
        min_fps (int, optional): The fps of a single slice. Defaults to 25.
    """
    error=[]
    info_error=[]
    for paths,dirs,files in os.walk(root):
        if len(dirs)==0 and len(files)>=threshold:
            try:
                series_num=[]
                pos_z=[]
                for i in range(len(files)):
                    try:
                        info=pydicom.dcmread(os.path.join(paths,files[i]))
                        pz=float(list(info['0020','0032'].value)[-1])
                        sn=info['0020','0011'].value
                        if pz not in pos_z:
                            pos_z.append(pz)
                        if sn not in series_num:
                            series_num.append(sn)
                    except:
                        print(f'error with {os.path.join(paths,files[i])}')
                        os.remove(os.path.join(paths,files[i]))
                        info_error.append(os.path.join(paths,files[i]))
                pos_z.sort(reverse=True)
                # if len(series_num)<len(files)/min_fps:
                if len(series_num)<len(pos_z):
                    print(f'{paths} own {len(files)} dicom files,\
                            \nsingle slice fps set as {min_fps},\
                            \nbut only {len(series_num)} series given by heads of dicom files,\
                            \nthey are {series_num}\
                            \ntheir physical z-axis position: {pos_z}')
                    for i in range(len(files)):
                        dicom_path=os.path.join(paths,files[i])
                        if dicom_path in info_error:
                            continue
                        info=pydicom.dcmread(dicom_path)
                        if expand_mod==False:
                            try:
                                info.SeriesNumber=int(info['0020','9057'].value)
                            except KeyError:
                                try:
                                    info.SeriesNumber=int(1+pos_z.index(float(list(info['0020','0032'].value)[-1])))
                                except:
                                    # print('set error')
                                    pass
                                error.append(os.path.join(paths,files[i]))
                                # print(info.SeriesNumber)
                        else:
                            # print(f'Expand mode: {expand_mod}')
                            try:
                                info.SeriesNumber=int(1+pos_z.index(float(list(info['0020','0032'].value)[-1])))
                            except:
                                # print('set error')
                                pass
                            error.append(os.path.join(paths,files[i]))
                        info.save_as(os.path.join(paths,files[i]))
                        # info['0020','0011']=instack_id
            except:
                print(f'unknow error occured with {paths}')
    return error,info_error 
# fix_seriesnum(root='E:/BaiduNetdiskDownload/CAD/')
# fix_seriesnum(root='/Users/airskcer/Downloads/HCM_Resampled_dicom-20230112/',min_fps=1,threshold=1,expand_mod=False)
# fix_seriesnum(root='/Users/airskcer/Downloads/RCM_Resampled_dicom-20230107/11016694_WANG_WEN_HOU/',min_fps=1,threshold=1,expand_mod=False)
# fix_seriesnum(root='E:/BaiduNetdiskDownload/DCM_Resampled_dicom-20221210/')
# fix_seriesnum(root='F:/resampled_dicom/CHD_Resampled_dicom-20221210')
# fix_seriesnum(root=r'E:\BaiduNetdiskDownload\DCM_Resampled_dcm\12113522_JIA_DE_FU\20220225\BTFE_BH_SA',threshold=20)

In [10]:
del_empty(root='/Users/airskcer/Downloads')
def del_ef(root='/Users/airskcer/Downloads'):
    for roots,dirs,files in os.walk(root):
        if len(dirs)==0 and 'EF' in roots or 'Argus' in roots:
        # if len(dirs) == 0 and ('PSIR_SSh_BTFE_TI250' in roots or 'PSIR_SSh_BTFE_TI300' in roots):
        # if len(dirs) == 0 and 'MAG_PSMDE SingleShot' in roots:
            print(roots,len(files))
            # shutil.rmtree(roots)
del_ef()
# del_empty(root='/Users/airskcer/Downloads')

## Find out abnormal LGE data according to fps

In [85]:
del_empty(root='/Users/airskcer/Downloads')
mac_bug(root='/Users/airskcer/Downloads/')
ds_store(root='/Users/airskcer/Downloads/')
root='/Users/airskcer/Downloads/LVNC_nifti-20221224/'
for roots,dirs,files in os.walk(root):
    if len(dirs)==0:
        file=exclude_seg_files(os.listdir(roots))
        if len(files)>=1 and 'LGE' in roots:
            for i in range(len(files)):
                fps=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,files[i]))).shape[0]
                if fps>3:
                    print(roots,files[i],fps)
                    if fps>10:
                        os.remove(os.path.join(roots,files[i]))
        if len(files)==0:
            print(roots)
            shutil.rmtree(roots)

## Save dicom data as nifti data, must be done AFTER dataset resample
For SAX data, bigger slice number means lower position(at the most time)

In [ ]:
def save_nifti(dataroot='./New_dicom/',output='./',fps=25,thres=4):
    """Convert dicom data files into nifti format, and resample its resolution
    Args:
        dataroot (str, optional): The root path of resampled data. Defaults to './New_dicom/'.
        output (str, optional): The output path of generated nfiti data. Defaults to './'.
        fps (int, optional): The fps of a single slice(SAX/LAX4CH). Defaults to 25.
    """
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            slice_map={}
            file_info={}
            for i in range(len(files)):
                info=pydicom.dcmread(os.path.join(roots,files[i]))
                file_info[files[i]]=info
                sn=info.SeriesNumber
                if sn not in list(slice_map.keys()):
                    slice_map[sn]=[files[i]]
                else:
                    slice_map[sn].append(files[i])
            slices=list(slice_map.keys())
            for i in range(len(slices)):
                dcms=slice_map[slices[i]]
                pats_name=str(file_info[dcms[0]]['0010','0010'].value).replace(' ','_')
                pats_id=file_info[dcms[0]]['0010','0020'].value
                filetag=str(file_info[dcms[0]]['0008','103e'].value)
                print(f'filetag:{filetag}')
                if ('BH' in filetag or '4CH' in filetag or '4ch' in filetag or '4 CH' in filetag or 'TFE' in filetag or 'SecondaryCapture' in filetag or '1'==filetag) and ('SA' not in filetag and 'sa' not in filetag and 'shot' not in filetag and '8slices' not in filetag and 'LVOT' not in filetag):
                    if len(dcms)%fps==0:
                        temp_folder=os.path.join(output,'4ch_temp')
                        savepath=os.path.join(output,'4CH_data',f'{pats_id}_{pats_name}')
                        filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                    else:
                        temp_folder=os.path.join(output,'4ch_lge_temp')
                        savepath=os.path.join(output,'4CH_LGE_data',f'{pats_id}_{pats_name}')
                        filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                    for j in range(len(dcms)):
                        if not os.path.exists(temp_folder):
                            os.makedirs(temp_folder)
                        shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                    if not os.path.exists(savepath):
                        os.makedirs(savepath)
                    # else:
                    #     os.remove(os.path.join(savepath,os.listdir(savepath)[0]))
                    try:
                        dcm2nii(temp_folder,os.path.join(savepath,filename))
                    except:
                        pass
                    # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                    print(f'{filename} saved into {savepath}')
                    shutil.rmtree(temp_folder)
                elif 'SA' in filetag or 'sa' in filetag or '2'==filetag or 'shot' in filetag or 'PSIR_TFE 8slices'==filetag:
                    """
                    Bigger slice ID, lower z-axis position
                    """
                    if len(dcms)%fps==0:
                        temp_folder=os.path.join(output,'sax_temp')
                        savepath=os.path.join(output,'SAX_data',f'{pats_id}_{pats_name}')
                        filename=f'slice_{slices[i]}.nii.gz'
                    else:
                        temp_folder=os.path.join(output,'sax_lge_temp')
                        savepath=os.path.join(output,'SAX_LGE_data',f'{pats_id}_{pats_name}')
                        filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                    for j in range(len(dcms)):
                        if not os.path.exists(temp_folder):
                            os.makedirs(temp_folder)
                        shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                    if not os.path.exists(savepath):
                        os.makedirs(savepath)
                    try:
                        dcm2nii(temp_folder,os.path.join(savepath,filename))
                    except:
                        pass
                    # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                    print(f'{filename} saved into {savepath}')
                    shutil.rmtree(temp_folder)
    temps=['4ch_temp','4ch_lge_temp','sax_temp','sax_lge_temp']
    for i in range(len(temps)):
        try:
            shutil.rmtree(os.path.join(output,temps[i]))
        except:
            pass

# save_nifti(dataroot='./test2/HCM_new/12162262_WANG_JIAN_LIANG/',output='./test2/HCM_nfiti')
# save_nifti(dataroot='F:/resampled_dicom/DCM_Resampled_dcm-20221129/12093484_MA_QUAN_BAO/20220105/PS_PSMDE SPGR SA/',output='E:/nifti_original_space/DCM_nifti-20221129/SAX_LGE_data/',fps=25,thres=4)
# save_nifti(dataroot='E:/BaiduNetdiskDownload/NP_Resampled_dicom-20221212/12083851_WANG_WEN_HOU/20211210/Cine FIESTA SA/',output='E:/BaiduNetdiskDownload/NP_nifti-20221212/',fps=25,thres=4)
# fix_seriesnum(root=r'E:\BaiduNetdiskDownload\CAD_Resampled_dicom\12027538_LI_BAO_MING\20210309\Cine FIESTA SA',threshold=30)
# save_nifti2(dataroot='E:/BaiduNetdiskDownload/ARVC_2_Resampled_dicom/11589224_ZHANG_MING_HAO/20160126/B-TFE_BH/',output='E:/VST_fusion/ARVC_2_nifti/',fps=30,thres=50)
# save_nifti2(dataroot='E:/BaiduNetdiskDownload/RCM_Resampled_dicom-20221216/11894742_SUN_JIAN_LIN/20210705/4CH_LGE/',output='E:/BaiduNetdiskDownload/RCM_nifti-20221216',fps=25,thres=4)
# save_nifti2(dataroot='E:/BaiduNetdiskDownload/HCM_Resampled_dicom/12066703_CHEN_BAI_GANG/20210810/PSIR_TFE 8slices_LGE/',output='E:/BaiduNetdiskDownload/HCM_nifti',fps=25,thres=4)
# save_nifti2(dataroot='E:/BaiduNetdiskDownload/HCM_Resampled_dcm/12145504_SUN_QING_HE/20220808/cine_trufi_retro_4ch/',output='E:/BaiduNetdiskDownload/HCM_nifti',fps=25)

## Save LGE

In [8]:
def save_lge(dataroot='./New_dicom/',output='./',fps=25,thres=100):
    """Convert dicom data files into nifti format, and resample its resolution
    Args:
        dataroot (str, optional): The root path of resampled data. Defaults to './New_dicom/'.
        output (str, optional): The output path of generated nfiti data. Defaults to './'.
        fps (int, optional): The fps of a single slice(SAX/LAX4CH). Defaults to 25.
    """
    # try:
    #     shutil.rmtree(os.path.join(output,'SAX_LGE_data',dataroot.split('/')[-1]))
    # except:
    #     pass
    error=[]
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            slice_map={}
            file_info={}
            files=exclude_seg_files(files)
            try:
                for i in range(len(files)):
                    info=pydicom.dcmread(os.path.join(roots,files[i]))
                    file_info[files[i]]=info
                    sn=info.SeriesNumber
                    if sn not in list(slice_map.keys()):
                        slice_map[sn]=[files[i]]
                    else:
                        slice_map[sn].append(files[i])
                slices=list(slice_map.keys())
                for i in range(len(slices)):
                    dcms=slice_map[slices[i]]
                    pats_name=str(file_info[dcms[0]]['0010','0010'].value).replace(' ','_')
                    pats_id=file_info[dcms[0]]['0010','0020'].value
                    filetag=str(file_info[dcms[0]]['0008','103e'].value)
                    print(filetag)
                    # if ('BH' in filetag or '4CH' in filetag or '4 CH' in filetag or 'TFE' in filetag or 'SecondaryCapture' in filetag or '1'==filetag) and ('SA' not in filetag and 'sa' not in filetag and 'shot' not in filetag and '8slices' not in filetag and 'LVOT' not in filetag):
                    if len(files) < thres:
                        print(f'fps: {len(dcms)}')
                        # if len(dcms)%fps==0:
                        if len(dcms)>=10:
                            # temp_folder=os.path.join(output,'4ch_temp')
                            # savepath=os.path.join(output,'4CH_data',f'{pats_id}_{pats_name}')
                            # filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            continue
                        else:
                            temp_folder=os.path.join(output,'4ch_lge_temp')
                            savepath=os.path.join(output,'4CH_LGE_data',f'{pats_id}_{pats_name}')
                            filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            # continue
                        for j in range(len(dcms)):
                            if not os.path.exists(temp_folder):
                                os.makedirs(temp_folder)
                            shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                        if not os.path.exists(savepath):
                            os.makedirs(savepath)
                        try:
                        # if True:
                            dcm2nii(temp_folder,os.path.join(savepath,filename))
                            # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                            print(f'{filename} saved into {savepath}')
                        except:
                            pass
                        shutil.rmtree(temp_folder)
                    # elif 'SA' in filetag or 'sa' in filetag or '2'==filetag or 'shot' in filetag or 'PSIR_TFE 8slices'==filetag:
                    elif len(files)>=thres:
                        # continue
                        print(f'fps: {len(dcms)}')
                        """
                        Bigger slice ID, lower z-axis position
                        """
                        # if len(dcms)%fps==0:
                        if len(dcms)>=25:
                        # if True:
                            # temp_folder=os.path.join(output,'sax_temp')
                            # savepath=os.path.join(output,'SAX_data',f'{pats_id}_{pats_name}')
                            # filename=f'slice_{slices[i]}.nii.gz'
                            continue
                        else:
                            temp_folder=os.path.join(output,'sax_lge_temp')
                            savepath=os.path.join(output,'SAX_LGE_data',f'{pats_id}_{pats_name}')
                            filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            # continue
                        for j in range(len(dcms)):
                            if not os.path.exists(temp_folder):
                                os.makedirs(temp_folder)
                            shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                        if not os.path.exists(savepath):
                            os.makedirs(savepath)
                        try:
                            dcm2nii(temp_folder,os.path.join(savepath,filename))
                            # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                            print(f'{filename} saved into {savepath}')
                        except:
                            pass
                        shutil.rmtree(temp_folder)
            except:
                print(f'unknow error occured with {roots}')
                error.append(roots)
    temps=['4ch_temp','4ch_lge_temp','sax_temp','sax_lge_temp']
    for i in range(len(temps)):
        try:
            shutil.rmtree(os.path.join(output,temps[i]))
        except:
            pass
    return error

lge_error=save_lge(dataroot='/Users/airskcer/Downloads/HCM_Resampled_dicom-20230112/',output='/Users/airskcer/Downloads/HCM_nifti-20230112/', fps=25, thres=4)
# save_lge(dataroot='/Users/airskcer/Downloads/RCM_Resampled_dicom-20230107/11241263_DU_GANG_JUN/',output='/Users/airskcer/Downloads/RCM_nifti-20230107/',fps=25,thres=4)

## Save CINE

In [20]:
def save_cine(dataroot='./New_dicom/',output='./',fps=25,thres=60):
    """Convert dicom data files into nifti format, and resample its resolution
    Args:
        dataroot (str, optional): The root path of resampled data. Defaults to './New_dicom/'.
        output (str, optional): The output path of generated nfiti data. Defaults to './'.
        fps (int, optional): The fps of a single slice(SAX/LAX4CH). Defaults to 25.
    """
    error=[]
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            try:
                slice_map={}
                file_info={}
                files=exclude_seg_files(files)
                for i in range(len(files)):
                    info=pydicom.dcmread(os.path.join(roots,files[i]))
                    file_info[files[i]]=info
                    sn=info.SeriesNumber
                    if sn not in list(slice_map.keys()):
                        slice_map[sn]=[files[i]]
                    else:
                        slice_map[sn].append(files[i])
                slices=list(slice_map.keys())
                for i in range(len(slices)):
                    dcms=slice_map[slices[i]]
                    pats_name=str(file_info[dcms[0]]['0010','0010'].value).replace(' ','_')
                    pats_id=file_info[dcms[0]]['0010','0020'].value
                    filetag=str(file_info[dcms[0]]['0008','103e'].value)
                    print(filetag)
                    # if ('BH' in filetag or '4CH' in filetag or '4 CH' in filetag or 'TFE' in filetag or 'SecondaryCapture' in filetag or '1'==filetag) and ('SA' not in filetag and 'sa' not in filetag and 'shot' not in filetag and '8slices' not in filetag and 'LVOT' not in filetag):
                    if len(files)<thres and 'LVOT' not in filetag:
                        print(f'fps: {len(dcms)}')
                        # if len(dcms)%fps==0:
                        if len(dcms)>=10:
                            temp_folder=os.path.join(output,'4ch_temp')
                            savepath=os.path.join(output,'4CH_data',f'{pats_id}_{pats_name}')
                            filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            # continue
                        else:
                            # temp_folder=os.path.join(output,'4ch_lge_temp')
                            # savepath=os.path.join(output,'4CH_LGE_data',f'{pats_id}_{pats_name}')
                            # filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            continue
                        for j in range(len(dcms)):
                            if not os.path.exists(temp_folder):
                                os.makedirs(temp_folder)
                            shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                        if not os.path.exists(savepath):
                            os.makedirs(savepath)
                        try:
                        # if True:
                            dcm2nii(temp_folder,os.path.join(savepath,filename))
                            # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                            print(f'{filename} saved into {savepath}')
                        except:
                            pass
                        shutil.rmtree(temp_folder)
                    # elif 'SA' in filetag or 'sa' in filetag or '2'==filetag or 'shot' in filetag or 'PSIR_TFE 8slices'==filetag:
                    elif len(files)>=thres:
                        print(f'fps: {len(dcms)}')
                        """
                        Bigger slice ID, lower z-axis position
                        """
                        # if len(dcms)%fps==0:
                        # if len(dcms)>=25:
                        if True:
                            temp_folder=os.path.join(output,'sax_temp')
                            savepath=os.path.join(output,'SAX_data',f'{pats_id}_{pats_name}')
                            filename=f'slice_{slices[i]}.nii.gz'
                            # continue
                        else:
                            # temp_folder=os.path.join(output,'sax_lge_temp')
                            # savepath=os.path.join(output,'SAX_LGE_data',f'{pats_id}_{pats_name}')
                            # filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            continue
                        for j in range(len(dcms)):
                            if not os.path.exists(temp_folder):
                                os.makedirs(temp_folder)
                            shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                        if not os.path.exists(savepath):
                            os.makedirs(savepath)
                        try:
                            dcm2nii(temp_folder,os.path.join(savepath,filename))
                            # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                            print(f'{filename} saved into {savepath}')
                        except:
                            pass 
                        shutil.rmtree(temp_folder)
            except:
                print(f'unknow error occured with {roots}')
                error.append(roots)
    temps=['4ch_temp','4ch_lge_temp','sax_temp','sax_lge_temp']
    for i in range(len(temps)):
        try:
            shutil.rmtree(os.path.join(output,temps[i]))
        except:
            pass
    return error
cine_error=save_cine(dataroot='/Users/airskcer/Downloads/HCM_Resampled_dicom-20230112/',output='/Users/airskcer/Downloads/HCM_nifti-20230112/',fps=25,thres=80)
# save_cine(dataroot='/Users/airskcer/Downloads/RCM_Resampled_dicom-20230107/11124028_SHI_KE_GANG/',output='/Users/airskcer/Downloads/RCM_nifti-20230107/',fps=25,thres=60)

In [ ]:
# path='/Users/airskcer/Downloads/check/'
# pats=os.listdir(path)
# pats=exclude_seg_files(pats)
# for i in range(len(pats)):
#     id=int(pats[i].split('.jpg')[0].split('_')[-1])
#     pat_name=pats[i].split('.jpg')[0].replace(f'_{id}','')
#     save_cine(dataroot=f'/Users/airskcer/Downloads/HCM_Resampled_dicom-20230102/{pat_name}',output='/Users/airskcer/Downloads/HCM_nifti-20230102/',thres=60)
# for roots,dirs,files in os.walk('/Users/airskcer/Downloads/HCM_nifti-20230102/4CH_data/'):
#     if len(dirs)==0:
#         files=exclude_seg_files(files)
#         for i in range(len(files)):
#             if files[i].replace('.nii.gz','.jpg') in pats:
#                 print(files[i])
#                 os.remove(os.path.join(roots,files[i]))

## Check data lost

In [21]:
nifti_path='/Users/airskcer/Downloads/HCM_nifti-20230112/4CH_data/'
dcm_path='/Users/airskcer/Downloads/HCM_Resampled_dicom-20230112/'
dcms=os.listdir(dcm_path)
niftis=os.listdir(nifti_path)
error=[]
for i in range(len(niftis)):
    if niftis[i].startswith('._'):
        os.remove(os.path.join(nifti_path,niftis[i]))
print(len(niftis),len(dcms))
for i in range(len(dcms)):
    if dcms[i] not in niftis:
        print(dcms[i])
        error.append(os.path.join(dcm_path,dcms[i]))
# print(error)11

672 679
11935261_PEI_YUN_PENG
11868487_LIU_FENG_LAN
12104331_ZHONG_HUI
12096792_HOU_XIAO_LI
12103375_MA_YI_FEN
11865661_WU_CUI_LAN
11266381_LUO_CHANG_SHENG


In [18]:
ds_store(root='/Users/airskcer/Downloads/')
del_empty(root='/Users/airskcer/Downloads/')

## Divide 4CH_LGE and SAX_LGE into different folders

In [8]:
def fix_4ch_lge(path=r'E:\BaiduNetdiskDownload\ARVC_1\11235202_HE_HUI',fps=25,thres=4):
    for roots,dirs,files in os.walk(path):
        if len(dirs)==0:
            size={}
            if len(files)%fps!=0 and len(files)<50:
                for i in range(len(files)):
                    sd=pydicom.dcmread(os.path.join(roots,files[i])).pixel_array.shape
                    if sd in size.keys():
                        size[sd].append(os.path.join(roots,files[i]))
                    else:
                        size[sd]=[os.path.join(roots,files[i])]
                # print(size)
                # if len(size.keys())>1:
                #     print(roots)
                if len(size.keys())==2:
                    savepath=roots.replace(roots.split('/')[-1],'4CH_LGE')
                    # print(savepath)
                    try:
                        os.makedirs(savepath)
                    except:
                        pass
                    f_key=None
                    num=len(files)
                    for key in size.keys():
                        if len(size[key])<num:
                            f_key=key
                            num=len(size[key])
                    for i in range(len(size[f_key])): 
                        shutil.move(size[key][i],savepath)
                        print(f'{size[key][i]} moved to {savepath}')
# for i in range(len(error)):
#     save_nifti2(dataroot=error[i],output='E:/BaiduNetdiskDownload/ARVC_1_nifti',fps=25,thres=4)
# fix_4ch_lge(path='E:/BaiduNetdiskDownload/CHD_Resampled_dicom-20221210/')
fix_4ch_lge(path='/Users/airskcer/Downloads/HCM_Resampled_dicom-20230112/')
# fix_4ch_lge(path='/Users/airskcer/Downloads/RCM_Resampled_dicom-20230107/')


AttributeError: Unable to convert the pixel data: one of Pixel Data, Float Pixel Data or Double Float Pixel Data must be present in the dataset

## Divide 4CH and SAX into different folders

In [ ]:
def fix_4ch(path=r'E:\BaiduNetdiskDownload\ARVC_1\11235202_HE_HUI',fps=25,thres=30):
    for roots,dirs,files in os.walk(path):
        if len(dirs)==0:
            size={}
            if len(files)>thres:
                for i in range(len(files)):
                    sd=pydicom.dcmread(os.path.join(roots,files[i])).pixel_array.shape
                    if sd in size.keys():
                        size[sd].append(os.path.join(roots,files[i]))
                    else:
                        size[sd]=[os.path.join(roots,files[i])]
                # print(size)
                if len(size.keys())==2:
                    savepath=roots.replace(roots.split('/')[-1],'4CH')
                    print(savepath)
                    try:
                        os.makedirs(savepath)
                    except:
                        pass
                    for key in size.keys():
                        if len(size[key])<30:
                            print(len(size[key]))
                            for num in range(len(size[key])):
                                shutil.move(size[key][num],savepath)
                                print(f'{size[key][num]} moved to {savepath}')
# for i in range(len(error)):
#     # fix_4ch(path=error[i])
#     save_nifti2(dataroot=error[i],output='E:/BaiduNetdiskDownload/ARVC_2_nifti',fps=25,thres=50)
# fix_4ch(path='E:/BaiduNetdiskDownload/CHD_Resampled_dicom-20221210/')
fix_4ch(path='/Users/airskcer/Downloads/DCM_Resampled_dicom-20230108/')
# fix_4ch(path='/Users/airskcer/Downloads/RCM_Resampled_dicom-20230107/')

## Find out the error with 4ch/sax data

In [10]:
import pandas as pd

def find_error(nifti_path='E:/BaiduNetdiskDownload/PAH_nifti',niftis=[],cine_fps=25,max_slices_sax=13,max_slices_lax=1):
    '''
    ## Find out: 
        - The lost data
        - Multi slices of 4CH CINE/LGE
        - Error with cine_fps

    Automatically exclude the segmentation files
    '''
    chart=[]
    for i in range(len(niftis)):
        try:
            note={}
            info=niftis[i]
            note['NAME']=niftis[i]
            note['MOD']=nifti_path.split('/')[-1]
            # print(niftis[i])
            try:
                sax_file=os.path.join(nifti_path,'SAX_data',niftis[i])
                '''Remove segmentation files'''
                slice_files=exclude_seg_files(os.listdir(sax_file))

                if len(slice_files)>=max_slices_sax:
                    info+=f' Multiple_sax_cine_slice_{len(slice_files)}'
                    note['SAX_CINE']=f'{len(slice_files)} slices'
                sax_file=os.path.join(sax_file,slice_files[0])
                sax_data=sitk.GetArrayFromImage(sitk.ReadImage(sax_file))
                if len(sax_data)!=cine_fps:
                    info+=f" {len(sax_data)}fps-sax_cine"
                    note['SAX_CINE']=f'{len(sax_data)}fps'
            except FileNotFoundError:
                info+=' No_sax_cine'
                note['SAX_CINE']='No_data'
            try:
                lax_file=os.path.join(nifti_path,'4CH_data',niftis[i])
                '''Remove segmentation files'''
                slice_files=exclude_seg_files(os.listdir(lax_file))

                if len(slice_files)>max_slices_lax:
                    info+=f' Multiple_4ch_cine_slice_{len(slice_files)}'
                    note['LAX4CH_CINE']=f'{len(slice_files)} slices'
                lax_file=os.path.join(lax_file,slice_files[0])
                lax_data=sitk.GetArrayFromImage(sitk.ReadImage(lax_file))
                if len(lax_data)!=cine_fps:
                    info+=f' {len(lax_data)}fps-4ch_cine'
                    note['LAX4CH_CINE']=f'{len(lax_data)}fps'
            except FileNotFoundError:
                info+=' No_4ch_cine'
                note['LAX4CH_CINE']='No_data'
            try:
                sax_file=os.path.join(nifti_path,'SAX_LGE_data',niftis[i])
                '''Remove segmentation files'''
                slice_files=exclude_seg_files(os.listdir(sax_file))
                
                if len(slice_files)>=max_slices_sax:
                    info+=f' Multiple_sax_lge_slice_{len(slice_files)}'
                    note['SAX_LGE']=f'{len(slice_files)} slices'
                sax_file=os.path.join(sax_file,slice_files[0])
                sax_data=sitk.GetArrayFromImage(sitk.ReadImage(sax_file))
            except FileNotFoundError:
                info+=' No_sax_lge'
                note['SAX_LGE']='No_data'
            try:
                lax_file=os.path.join(nifti_path,'4CH_LGE_data',niftis[i])
                '''Remove segmentation files'''
                slice_files=exclude_seg_files(os.listdir(lax_file))
                
                if len(slice_files)>max_slices_lax:
                    info+=f' Multiple_4ch_lge_slice_{len(slice_files)}'
                    note['LAX4CH_LGE']=f'{len(slice_files)} slices'
                lax_file=os.path.join(lax_file,slice_files[0])
                lax_data=sitk.GetArrayFromImage(sitk.ReadImage(lax_file))
            except FileNotFoundError:
                info+=' No_4ch_lge'
                note['LAX4CH_LGE']='No_data'
            if info!=niftis[i]:
                print(f"{info}")
                chart.append(note)
        except:
            print(f'error occured with {nifti_path} {niftis[i]}')
    return chart
# mac_bug(root='/Volumes/SSD/HHD_nifti-20221217/')
# del_empty(root='/Users/airskcer/Downloads/CAD_nifti-20221222/')
chart=find_error(nifti_path='/Users/airskcer/Downloads/DCM_nifti-20230108/',niftis=os.listdir('/Users/airskcer/Downloads/DCM_Resampled_dicom-20230108/'))
# chart=find_error(nifti_path='E:/BaiduNetdiskDownload/NP_nifti-20221212/',niftis=os.listdir('E:/BaiduNetdiskDownload/NP_Resampled_dicom-20221212'))

11931122_HU_XIONG Multiple_4ch_lge_slice_2
11092274_MENG_XIAN_MIN Multiple_4ch_lge_slice_2
11902554_MENG_GUANG_JIAN Multiple_4ch_lge_slice_10
11921730_MENG_WEN_LONG No_sax_lge Multiple_4ch_lge_slice_2
11868434_LI_FAN No_sax_lge
11930721_HUANG_SEN Multiple_4ch_lge_slice_2
11941747_LIU_HONG_ZHUO Multiple_4ch_lge_slice_2
11923656_LI_TAO Multiple_4ch_lge_slice_8
11913807_ZHANG_JING Multiple_4ch_lge_slice_2
11907261_DONG_SHU_JUN Multiple_4ch_lge_slice_6
11878939_ZHANG_LEI Multiple_4ch_lge_slice_2
11914281_ZHANG_JIAN_JUN Multiple_4ch_lge_slice_2
11897732_WANG_CHUN_WEI Multiple_4ch_lge_slice_2
11905737_LIU_SONG Multiple_4ch_lge_slice_4
11911404_HUO_LI_WEI Multiple_4ch_lge_slice_2
11913199_WEN_LU_XIANG Multiple_4ch_lge_slice_6
11913234_ZHAO_HONG_MEI Multiple_4ch_lge_slice_2
11875701_YU_HUA_GUO Multiple_4ch_lge_slice_2
11911925_QI_BAO_GUI Multiple_4ch_lge_slice_2
11951231_SONG_ZHEN Multiple_4ch_lge_slice_4
11878246_GONG_FANG_ZHEN Multiple_4ch_lge_slice_6
11515909_LI_RUI_FENG Multiple_4ch_lge_sl

In [11]:
chart=pd.DataFrame(chart)
chart.to_csv('./fuwai_dataset/DCM_20230108/error.csv',index=False)

## HE_LGE

In [ ]:
def hist(imgs):
    new_imgs=[]
    for i in range(len(imgs)):
        img = imgs[i]
        # img = np.array(img)
        img=255.0*(img-np.max(img))/(np.max(img)-np.min(img))
        img = img.astype(np.uint8)
        img=cv2.equalizeHist(img)
        img = img.astype(np.uint8)
        new_imgs.append(img)
    return np.array(new_imgs)
def LGE_HE(dataroot='./lge/4CH_LGE_data/',mod='4ch',rename=True,output=''):
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            if mod=='4ch':
                data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,files[0])))
                # print(data.shape,files)
                if data.shape[0]>1 and data.shape[0]<10:
                    data=np.array([data[int((data.shape[0]+1)/2)]])
                elif data.shape[0]>10:
                    data=np.array([data[:,:,0]])
                # print(data.shape)
                # data=255.0*(data-np.max(data))/(np.max(data)-np.min(data))
                if rename:
                    # sitk.WriteImage(sitk.GetImageFromArray(data),os.path.join(roots,'old_lge.nii.gz'))
                    pass
                    # os.remove(os.path.join(roots,files[0]))
                he_data=hist(data)
                data=255.0*(data-np.min(data))/(np.max(data)-np.min(data))
                data.astype(np.uint8)
                # print(roots.split(dataroot))
                try:
                    os.makedirs(os.path.join(output,roots.split(dataroot)[-1]))
                except:
                    pass
                cv2.imwrite(os.path.join(output,roots.split(dataroot)[-1],'old_lge.jpg'),data[0])
                cv2.imwrite(os.path.join(output,roots.split(dataroot)[-1],'new_he_lge.jpg'),he_data[0])
                # sitk.WriteImage(sitk.GetImageFromArray(he_data),os.path.join(roots,'new_lge_he.nii.gz'))
                # plt.figure()
                # plt.subplot(121)
                # plt.imshow(data[0],cmap='gray')
                # plt.subplot(122)
                # plt.imshow(he_data[0],cmap='gray')
                # plt.title(f'{np.max(he_data)},{np.min(he_data)};{np.max(data)},{np.min(data)}')

            if mod=='sax':
                data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,files[0])))
                # print(data.shape)
                if len(data.shape)==4:
                    
                    data=data[:,:,:,0]
                # print(data.shape,roots)
                if rename:
                    # sitk.WriteImage(sitk.GetImageFromArray(data),os.path.join(roots,'old_lge.nii.gz'))
                    pass
                    # os.remove(os.path.join(roots,files[0]))
                he_data=hist(data)
                for i in range(len(data)):
                    try:
                        os.makedirs(os.path.join(output,roots.split(dataroot)[-1],'new_he_lge'))
                    except:
                        pass
                    try:
                        os.makedirs(os.path.join(output,roots.split(dataroot)[-1],'old_lge'))
                    except:
                        pass
                    data[i]=255.0*(data[i]-np.min(data[i]))/(np.max(data[i])-np.min(data[i]))
                    data.astype(np.uint8)
                    # os.remove(os.path.join(output,roots.split(dataroot)[-1],'old_lge.jpg'))
                    # os.remove(os.path.join(output,roots.split(dataroot)[-1],'new_he_lge.jpg'))
                    cv2.imwrite(os.path.join(output,roots.split(dataroot)[-1],'old_lge',f'img_{i+1:05d}.jpg'),data[i])
                    cv2.imwrite(os.path.join(output,roots.split(dataroot)[-1],'new_he_lge',f'img_{i+1:05d}.jpg'),he_data[i])
                # sitk.WriteImage(sitk.GetImageFromArray(he_data),os.path.join(roots,'new_lge_he.nii.gz'))

def LGE_HE2(dataroot='./New_dicom/',output='./',fps=25):
    """Convert dicom data files into nifti format, and resample its resolution
    Args:
        dataroot (str, optional): The root path of resampled data. Defaults to './New_dicom/'.
        output (str, optional): The output path of generated nfiti data. Defaults to './'.
        fps (int, optional): The fps of a single slice(SAX/LAX4CH). Defaults to 25.
    """
    temps=['4ch_temp','4ch_lge_temp','sax_temp','sax_lge_temp']
    for i in range(len(temps)):
        try:
            shutil.rmtree(os.path.join(output,temps[i]))
        except:
            pass
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            slice_map={}
            file_info={}
            for i in range(len(files)):
                info=pydicom.dcmread(os.path.join(roots,files[i]))
                file_info[files[i]]=info
                sn=info.SeriesNumber
                if sn not in list(slice_map.keys()):
                    slice_map[sn]=[files[i]]
                else:
                    slice_map[sn].append(files[i])
            slices=list(slice_map.keys())
            for i in range(len(slices)):
                dcms=slice_map[slices[i]]
                pats_name=str(file_info[dcms[0]]['0010','0010'].value).replace(' ','_')
                pats_id=file_info[dcms[0]]['0010','0020'].value
                filetag=str(file_info[dcms[0]]['0008','103e'].value)
                # print(filetag)
                # if ('BH' in filetag or '4CH' in filetag or '4 CH' in filetag or 'TFE' in filetag or 'SecondaryCapture' in filetag or '1'==filetag) and ('SA' not in filetag and 'sa' not in filetag and 'shot' not in filetag and '8slices' not in filetag and 'LVOT' not in filetag):
                if len(files)<4:
                    if len(dcms)%fps==0:
                        temp_folder=os.path.join(output,'4ch_temp')
                        savepath=os.path.join(output,'4CH_data',f'{pats_id}_{pats_name}')
                        filename=f'{pats_id}_{pats_name}.nii.gz'
                    else:
                        temp_folder=os.path.join(output,'4ch_lge_temp')
                        savepath=os.path.join(output,'4CH_LGE_data',f'{pats_id}_{pats_name}')
                        filename=f'{pats_id}_{pats_name}.nii.gz'
                    for j in range(len(dcms)):
                        if not os.path.exists(temp_folder):
                            os.makedirs(temp_folder)
                        shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                    if not os.path.exists(savepath):
                        os.makedirs(savepath)
                    dcm2nii(temp_folder,os.path.join(savepath,'old_lge.nii.gz'))
                    dcm_temp=os.listdir(temp_folder)
                    for index in range(len(dcm_temp)):
                        data=pydicom.dcmread(os.path.join(temp_folder,dcm_temp[index]))
                        img=data.pixel_array
                        shape=img.shape
                        if len(img.shape)==3:
                            img=img[:,:,0]
                        img=255.0*((img-np.max(img))/(np.max(img)-np.min(img)))
                        img=img.astype(np.uint8)
                        img=cv2.equalizeHist(img)
                        # img=255.0*((img-np.max(img))/(np.max(img)-np.min(img)))
                        if len(data.pixel_array.shape)==3:
                            img=img.astype(np.uint8)
                        else:
                            img=img.astype(np.uint16)
                        data.PixelData=img.tobytes()
                        pydicom.dcmwrite(os.path.join(temp_folder,dcm_temp[index]),data)
                    dcm2nii(temp_folder,os.path.join(savepath,'new_he_lge.nii.gz'))
                    # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                    print(f'{filename} saved into {savepath}')
                    shutil.rmtree(temp_folder)
                # elif 'SA' in filetag or 'sa' in filetag or '2'==filetag or 'shot' in filetag or 'PSIR_TFE 8slices'==filetag:
                elif len(files)>7:
                    """
                    Bigger slice ID, lower z-axis position
                    """
                    if len(dcms)%fps==0:
                        temp_folder=os.path.join(output,'sax_temp')
                        savepath=os.path.join(output,'SAX_data',f'{pats_id}_{pats_name}')
                        filename=f'slice_{slices[i]}.nii.gz'
                    else:
                        temp_folder=os.path.join(output,'sax_lge_temp')
                        savepath=os.path.join(output,'SAX_LGE_data',f'{pats_id}_{pats_name}')
                        filename=f'{pats_id}_{pats_name}.nii.gz'
                    for j in range(len(dcms)):
                        if not os.path.exists(temp_folder):
                            os.makedirs(temp_folder)
                        shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                    if not os.path.exists(savepath):
                        os.makedirs(savepath)
                    dcm2nii(temp_folder,os.path.join(savepath,'old_lge.nii.gz'))
                    dcm_temp=os.listdir(temp_folder)
                    for index in range(len(dcm_temp)):
                        data=pydicom.dcmread(os.path.join(temp_folder,dcm_temp[index]))
                        img=data.pixel_array
                        shape=img.shape
                        if len(img.shape)==3:
                            img=img[:,:,0]
                        img=255.0*((img-np.max(img))/(np.max(img)-np.min(img)))
                        img=img.astype(np.uint8)
                        img=cv2.equalizeHist(img)
                        # img=255.0*((img-np.max(img))/(np.max(img)-np.min(img)))
                        if len(data.pixel_array.shape)==3:
                            img=img.astype(np.uint8)
                        else:
                            img=img.astype(np.uint16)
                        data.PixelData=img.tobytes()
                        pydicom.dcmwrite(os.path.join(temp_folder,dcm_temp[index]),data)
                    dcm2nii(temp_folder,os.path.join(savepath,'new_he_lge.nii.gz'))
                    # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                    print(f'{filename} saved into {savepath}')
                    shutil.rmtree(temp_folder)
    temps=['4ch_temp','4ch_lge_temp','sax_temp','sax_lge_temp']
    for i in range(len(temps)):
        try:
            shutil.rmtree(os.path.join(output,temps[i]))
        except:
            pass
def hist_equalization(img_array):
        histogram_array = np.bincount(img_array.flatten(), minlength=256)
        # normalize
        num_pixels = np.sum(histogram_array)
        histogram_array = histogram_array / num_pixels
        # cumulative histogram
        chistogram_array = np.cumsum(histogram_array)
        """
        STEP 2: Pixel mapping lookup table
        """
        transform_map = np.floor(255 * chistogram_array).astype(np.uint8)
        """
        STEP 3: Transformation
        """
        # flatten image array into 1D list
        img_list = list(img_array.flatten())
        # transform pixel values to equalize
        eq_img_list = [transform_map[p] for p in img_list]
        # reshape and write back into img_array
        eq_img_array = np.reshape(np.asarray(eq_img_list), img_array.shape)

        return np.uint8(np.array(eq_img_array))

def img2sitkdata(original_sitkdata: sitk.Image, img: np.ndarray):
    new_data = sitk.GetImageFromArray(img)
    new_data.SetDirection(original_sitkdata.GetDirection())
    # new_data.SetMetaData(original_sitkdata.GetMetaData())
    new_data.SetOrigin(original_sitkdata.GetOrigin())
    new_data.SetSpacing(original_sitkdata.GetSpacing())
    return new_data

def Getdis(imgpath='',img=None,output='',name='img.jpg',Tag='Normal',show=True):
    if imgpath!='':
        img=cv2.imread(imgpath)
    if len(img.shape)==3:
        img=img[:,:,0]
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plt.imshow(img,cmap='gray')
    plt.title(f'(Mean,STD): {np.mean(img),np.std(img)}\n{Tag}')
    plt.subplot(122)
    plt.hist(img.flatten(),bins=20,rwidth=0.9,density=True)
    plt.xlabel('Intensity')
    plt.ylabel('Proportion')
    plt.title('Intensity Distribution of the Image')
    if output!='':
        plt.savefig(os.path.join(output,name),bbox_inches='tight',pad_inches=0.5)
    if show==True:
        plt.show()
    # plt.clf()
    return 0

# ori=pydicom.dcmread('./old.dcm')
# data1=pydicom.dcmread('./he.dcm')
# data2=pydicom.dcmread('./he2.dcm')
# Getdis(img=ori.pixel_array)
# Getdis(img=data1.pixel_array)
# Getdis(img=data2.pixel_array)
# LGE_HE2(dataroot='./test2/CAD_new/',output='new_lge')
# data=pydicom.dcmread('./old.dcm')
# img=data.pixel_array
# img=255.0*((img-np.max(img))/(np.max(img)-np.min(img)))
# img=img.astype(np.uint8)
# # img=hist_equalization(img)
# img=cv2.equalizeHist(img)
# # img=255.0*((img-np.max(img))/(np.max(img)-np.min(img)))
# img=img.astype(np.uint16)
# print(np.max(img),np.min(img))
# data.PixelData=img.tobytes()
# pydicom.dcmwrite('./he.dcm',data)
# LGE_HE(dataroot='./lge/4CH_LGE_data/',mod='4ch',output='lge_img/4ch')
# LGE_HE(dataroot='./lge/SAX_LGE_data/',mod='sax',output='lge_img/sax')

In [ ]:
# for roots,dirs,files in os.walk('./test2/HCM_new/'):
#     if len(dirs)==0:
#         if len(files)%25==0 or len(files)==0:
#             shutil.rmtree(roots)
#             print(roots,len(files))

## Save cropped img data

In [ ]:
def lax4ch_img_data(path='./4CH_data/',output='./fuwai_4CH_data_img'):
    pats=os.listdir(path)
    for i in range(len(pats)):
        data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],f'{pats[i]}.nii.gz')))
        mask=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],'Segmentation.nii')))
        for j in range(len(mask)):
            if np.count_nonzero(mask[j]>0)>0:
                _,crop_pos=Rec_crop2D(mask,j)
                mask=mask[j]
                break
        savepath=os.path.join(output,pats[i])
        if not os.path.exists(savepath):
            os.makedirs(savepath)
        # plt.figure()
        # plt.subplot(121)
        # plt.imshow(data[0])
        # plt.subplot(122)
        # plt.imshow(mask)
        # plt.show()
        for j in range(len(data)):
            cv2.imwrite(os.path.join(savepath,f'img_{j+1:05d}.jpg'),norm_img(data[j][crop_pos[2]:crop_pos[3],crop_pos[0]:crop_pos[1]]))
        print(f'{pats[i]} images are saved in {savepath}')
    return 0
def sax_img_data(path='./SAX_data/',output='./fuwai_SAX_data_img'):
    pats=os.listdir(path)
    for i in range(len(pats)):
        files=os.listdir(os.path.join(path,pats[i]))
        slices=[]
        for k in range(len(files)):
            if 'slice' in files[k]:
                slices.append(int(files[k].split('.nii.gz')[0].split('_')[-1]))
        slices.sort()
        if len(slices)>3:
            slices=slices[len(slices)//3:len(slices)//3+3]
        mask=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],'Segmentation.nii')))
        for j in range(len(mask)):
            if np.count_nonzero(mask[j]>0)>0:
                _,crop_pos=Rec_crop2D(mask,j)
                mask=mask[j]
                break
        for k in range(len(slices)):
            data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],f'slice_{slices[k]}.nii.gz')))
            savepath=os.path.join(output,pats[i],f'{pos_map[k]}')
            if not os.path.exists(savepath):
                os.makedirs(savepath)
            # plt.figure()
            # plt.subplot(121)
            # plt.imshow(data[0])
            # plt.subplot(122)
            # plt.imshow(mask)
            # plt.show()
            for index in range(len(data)):
                cropped_img=data[index][crop_pos[2]:crop_pos[3],crop_pos[0]:crop_pos[1]]
                cropped_img=255.0*((cropped_img-np.min(cropped_img))/(np.max(cropped_img)-np.min(cropped_img)))
                cv2.imwrite(os.path.join(savepath,f'img_{index+1:05d}.jpg'),cropped_img)
            print(f'{pats[i]} slice {k} images are saved in {savepath}')
    return 0
def lge_img_data(path='./SAX_LGE_data/',output='./fuwai_sax_lge_data_img/'):
    pats=os.listdir(path)
    for i in range(len(pats)):
        data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],f'{pats[i]}.nii.gz')))
        mask=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],'Segmentation.nii')))
        for j in range(len(mask)):
            if np.count_nonzero(mask[j]>0)>0:
                _,crop_pos=Rec_crop2D(mask,j)
                mask=mask[j]
                data=data[j]
                # plt.figure()
                # plt.subplot(121)
                # plt.imshow(data)
                # plt.subplot(122)
                # plt.imshow(mask)
                # plt.show()
                savepath=os.path.join(output,pats[i])
                if not os.path.exists(savepath):
                    os.makedirs(savepath)
                cv2.imwrite(os.path.join(savepath,f'img_{1:05d}.jpg'),norm_img(data[crop_pos[2]:crop_pos[3],crop_pos[0]:crop_pos[1]]))
                print(f'{pats[i]} images are saved in {savepath}')
                break
    return 0

# lax4ch_img_data(path='./test2/HCM_me/4CH_data/',output='./test2/4ch_img_me')
# sax_img_data(path='./test2/HCM_me/SAX_data/',output='./test2/sax_img_me')
# lge_img_data(path='./test2/HCM_me/SAX_LGE_data/',output='./test2/lge_img_me')

## HE/HE Matching on croped ROI

In [ ]:
from skimage.exposure import match_histograms
def lge_roi_he(path='./test2/new_lge/4CH_LGE_data/',output='./test2/lge_roi_he',mod='4ch',oper='he',cropped=True,match_refer=''):
    pats=os.listdir(path)
    for i in range(len(pats)):
        data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],'old_lge.nii.gz')))
        if data.shape[-1]==3:
            if mod=='4ch':
                data=np.array([data[:,:,0]])
            elif mod=='sax':
                data=data[:,:,:,0]
        if cropped:
            mask=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path,pats[i],'Segmentation.nii')))
            crop_pos=crop(mask)
            data=data[:,crop_pos[2]:crop_pos[3],crop_pos[0]:crop_pos[1]]
        if match_refer!='':
            refer_img=norm_img(sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(match_refer,'old_lge.nii.gz')))[0])
            if cropped:
                refer_mask=crop(sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(match_refer,'Segmentation.nii'))))
                refer_img=norm_img(refer_img[refer_mask[2]:refer_mask[3],refer_mask[0]:refer_mask[1]])
        if mod=='4ch':
            data=data[(len(data)%2-1)]
            try:
                os.makedirs(os.path.join(output,pats[i]))
            except:
                pass
            cv2.imwrite(os.path.join(output,pats[i],'old_lge.jpg'),norm_img(data))
            if oper=='he':
                cv2.imwrite(os.path.join(output,pats[i],'new_he_lge.jpg'),norm_img(cv2.equalizeHist(norm_img(data))))
            elif oper=='match':
                cv2.imwrite(os.path.join(output,pats[i],'new_he_lge.jpg'),norm_img(match_histograms(norm_img(data),refer_img)))
        if mod=='sax':
            old_path=os.path.join(output,pats[i],'old_lge')
            new_he_path=os.path.join(output,pats[i],'new_he_lge')
            try:
                os.makedirs(old_path)
            except:
                pass
            try:
                os.makedirs(new_he_path)
            except:
                pass
            for i in range(len(data)):
                cv2.imwrite(os.path.join(old_path,f'img_{(i+1):05d}.jpg'),norm_img(data[i]))
                if oper=='he':
                    cv2.imwrite(os.path.join(new_he_path,f'img_{(i+1):05d}.jpg'),norm_img(cv2.equalizeHist(norm_img(data[i]))))
                elif oper=='match':
                    cv2.imwrite(os.path.join(new_he_path,f'img_{(i+1):05d}.jpg'),norm_img(match_histograms(norm_img(data[i]),refer_img)))

lge_roi_he(path='./test2/new_lge/4CH_LGE_data/',output='./test2/lge_roi_he_match/4CH',
            mod='4ch',cropped=False,oper='match',match_refer='./test2/new_lge/4CH_LGE_data/12163336_YU_HUI_BAO/')
lge_roi_he(path='./test2/new_lge/SAX_LGE_data/',output='./test2/lge_roi_he_match/SAX',
            mod='sax',cropped=False,oper='match',match_refer='./test2/new_lge/SAX_LGE_data/12163336_YU_HUI_BAO/')

In [ ]:

data1=sitk.GetArrayFromImage(sitk.ReadImage('./test2/new_lge/4CH_LGE_data/12143597_LIU_SHUANG_CAI/old_lge.nii.gz'))[0]
data2=sitk.GetArrayFromImage(sitk.ReadImage('./test2/new_lge/4CH_LGE_data/12151673_LI_JIAN_TING/old_lge.nii.gz'))[:,:,0]
plt.figure(figsize=(10,10))
plt.subplot(221)
plt.imshow(norm_img(data1),cmap='gray')
plt.subplot(222)
plt.imshow(norm_img(data2),cmap='gray')
plt.subplot(223)
img=match_histograms(data2,data1)
plt.imshow(img,cmap='gray')

## Generate ann

In [ ]:
def gen_ann():
    hcms=os.listdir('./test2/HCM_new/')
    cads=os.listdir('./test2/CAD_new/')
    with open('4ch_test_6cls.txt','w+')as f:
        root='/media/ssd2/yanran/Biobank-Test/fuwai_tests/test2/4ch_img'
        pats=os.listdir('./test2/4ch_img/')
        for i in range(len(pats)):
            cls= 0 if pats[i] in cads else 1
            f.write(f'{root}/{pats[i]} 25 {cls}\n')
    f.close()
    # with open('sax_4ch_lge_test.txt','w+')as f:
    #     pats=os.listdir('./test2/4ch_img_me/')
    #     for i in range(len(pats)):
    #         if pats[i] in hcms:
    #             f.write(f'/media/ssd2/yanran/Biobank-Test/fuwai_tests/test2/sax_img/{pats[i]}/mid /media/ssd2/yanran/Biobank-Test/fuwai_tests/test2/4ch_img/{pats[i]} /media/ssd2/yanran/Biobank-Test/fuwai_tests/test2/lge_img/{pats[i]} 25 1 1\n')
    #         else:
    #             f.write(f'/media/ssd2/yanran/Biobank-Test/fuwai_tests/test2/sax_img/{pats[i]}/mid /media/ssd2/yanran/Biobank-Test/fuwai_tests/test2/4ch_img/{pats[i]} /media/ssd2/yanran/Biobank-Test/fuwai_tests/test2/lge_img/{pats[i]} 25 1 0\n')
    # f.close()
# gen_ann()

## Get spacing info

In [27]:
import pandas as pd
from tqdm import tqdm

def get_data(path='./test2/HCM_new/'):
    """
    The get_data function returns a list of dictionaries, where each dictionary contains the patient's name, 
    patient ID number, and vendor. The function takes one argument: path to the directory containing all patients' 
    data folders. It will return a list of dictionaries with each dictionary containing information about one patient.
    
    :param path='./test2/HCM_new/': Specify the path to the directory containing all of your patients' dicom files
    :return: A list of dictionaries
    """
    pats=os.listdir(path)
    all_data=[]
    for i in range(len(pats)):
        pat_path=os.path.join(path,pats[i])
        pat_data={}
        for roots,dirs,files in os.walk(pat_path):
            if len(dirs)==0:
                for i in range(len(files)):
                    info=pydicom.dcmread(os.path.join(roots,files[i]))
                    pat_data['Name']=str(info['0010','0010'].value)
                    pat_data['ID']=info['0010','0020'].value
                    if len(files)%25==0:
                        pat_data['Vendor_cine']=str(info['0008','0070'].value)
                        if len(files)>100:
                            try:
                                pat_data['SAX_cine_z_spacing']=float(info['0018','0088'].value)
                            except:
                                pass
                            try:
                                pat_data['SAX_cine_xy_spacing']=list(info['0028','0030'].value)
                            except:
                                pass
                        else:
                            try:
                                pat_data['4CH_cine_z_spacing']=float(info['0018','0088'].value)
                            except:
                                pass
                            try:
                                pat_data['4CH_cine_xy_spacing']=list(info['0028','0030'].value)
                            except:
                                pass
                    # elif len(files):
                    else:
                        pat_data['Vendor_LGE']=info['0008','0070'].value
                        if len(files)>5:
                            try:
                                pat_data['SAX_LGE_z_spacing']=float(info['0018','0088'].value)
                            except:
                                pass
                            try:
                                pat_data['SAX_LGE_xy_spacing']=list(info['0028','0030'].value)
                            except:
                                pass
                        else:
                            try:
                                pat_data['4CH_LGE_z_spacing']=float(info['0018','0088'].value)
                            except:
                                pass
                            try:
                                pat_data['4CH_LGE_xy_spacing']=list(info['0028','0030'].value)
                            except:
                                pass
        print(pat_data)
        all_data.append(pat_data)
    return all_data
def get_info_from_nifti(nifti_path='E:/BaiduNetdiskDownload/DCM_nifti/',check_mode=True):
    niftis=['4CH_data','4CH_LGE_data','SAX_data','SAX_LGE_data']
    avail_pats=[]
    for i in range(len(niftis)):
        try:
            pats=os.listdir(os.path.join(nifti_path,niftis[i]))
            if len(pats)>len(avail_pats):
                avail_pats=pats
        except:
            pass
    print(f'{len(avail_pats)} patients available')
    all_info=[]
    
    bar=tqdm(range(len(avail_pats)),mininterval=1)
    for i in bar:
        pat_info=dict(name=avail_pats[i])
        for j in range(len(niftis)):
            sizes = []
            try:
                pat_folder=os.path.join(nifti_path,niftis[j],avail_pats[i])
                pat_files=exclude_seg_files(os.listdir(pat_folder))
                if check_mode:
                    for k in range(len(pat_files)):
                        data=sitk.ReadImage(os.path.join(pat_folder,pat_files[k]))
                        sizes.append(list(data.GetSpacing()))
                else:
                    data = sitk.ReadImage(os.path.join(pat_folder, pat_files[0]))
                spacing=data.GetSpacing()
                pat_info[f'{niftis[j]}_xyz']=spacing
            except:
                pass
            if check_mode:
                sizes=np.array(sizes)
                if np.all(np.std(sizes,axis=0))!=0:
                    print(pat_folder,sizes)
        all_info.append(pat_info)
        # break
    return all_info
# all_info=[]
# mods=os.listdir('E:/nifti_original_space/')
# for i in range(len(mods)):
#     folder=os.path.join('E:/nifti_original_space/',mods[i])
#     if os.path.isdir(folder):
#         print(folder)
#         niftis=['4CH_data','4CH_LGE_data','SAX_data','SAX_LGE_data']
#         avail_pats=[]
#         for j in range(len(niftis)):
#             try:
#                 pats=os.listdir(os.path.join(folder,niftis[j]))
#                 if len(pats)>len(avail_pats):
#                     avail_pats=pats
#             except:
#                 pass
        # all_info+=get_info_from_nifti(nifti_path=folder)
        # all_info+=find_error(nifti_path=folder,niftis=avail_pats)
# all_info=get_info_from_nifti(nifti_path='/Volumes/SSD/HHD_nifti-20221217/')
all_info=get_info_from_nifti(nifti_path='/Users/airskcer/Downloads/NP_nifti-20230109/',check_mode=False)
# all_info=find_error(nifti_path='E:/BaiduNetdiskDownload/RCM_nifti-20221216/',niftis=os.listdir('E:/BaiduNetdiskDownload/RCM_Resampled_dicom-20221216'))
# hcm_data1=get_data(path='E:/BaiduNetdiskDownload/HCM_Resampled_dcm/')
# cad_data=get_data(path='./test2/CAD_new/')
# all_data=pd.DataFrame(hcm_data+cad_data)
# all_data.to_excel('spacing_info.xlsx',index=False)
# data=pd.DataFrame(get_data(path='./test4/DCM_new/'))
# data.to_excel('./test4/spacing_info.xlsx',index=False)

151 patients available


100%|██████████| 151/151 [00:04<00:00, 33.13it/s]


In [28]:
pd.DataFrame(all_info).to_csv('./fuwai_dataset/NP_20230109/spacing_info.csv',index=False)

## Fix slices' frames

In [26]:
# path=os.path.join('E:/VST_fusion/ARVC_1_nifti/SAX_data/11097334_FENG_ZHEN',os.listdir('E:/VST_fusion/ARVC_1_nifti/SAX_data/11097334_FENG_ZHEN')[0])
# path='11097334_FENG_ZHEN.nii.gz'
# ori_data=sitk.ReadImage(path)
# spacing=list(ori_data.GetSpacing())
# fps=sitk.GetArrayFromImage(ori_data).shape[0]
# print(spacing,fps,fps/25)
# spacing[2]=fps/25
# data=resample_volume(Origin=path,output='output.nii.gz',new_spacing=spacing)
# print(sitk.GetArrayFromImage(data).shape)

def fix_single_file_fps(path,fps=25):
    data=sitk.ReadImage(path)
    spacing=list(data.GetSpacing())
    array=sitk.GetArrayFromImage(data)
    frames=len(array)
    if frames!=fps:
        new_space=spacing.copy()
        new_space[2]=frames/float(fps)
        resampled_data=resample_volume(Origin=path,output='',new_spacing=new_space)
        if sitk.GetArrayFromImage(resampled_data).shape[0]!=25:
            print(f'resample failure: {path}/{sitk.GetArrayFromImage(resampled_data).shape[0]} {spacing}/{new_space}')
        else:
            sitk.WriteImage(resampled_data,path)
def fix_fps(fps=25,nifti_path='E:/VST_fusion/ARVC_1_nifti/',fix=True):
    mods=['SAX_data','4CH_data']
    for i in range(len(mods)):
        for roots,dirs,files in os.walk(os.path.join(nifti_path,mods[i])):
            if len(dirs)==0:
                frame_list=[]
                files=exclude_seg_files(os.listdir(roots))
                for j in range(len(files)):
                    data=sitk.ReadImage(os.path.join(roots,files[j]))
                    spacing=list(data.GetSpacing())
                    array=sitk.GetArrayFromImage(data)
                    frames=len(array)
                    # print(frames)
                    frame_list.append(frames)
                    if frames!=fps:
                        print(f'{roots} {files[j]} {frames}')
                        if fix:
                            '''FIX FPS HERE'''
                            new_space=spacing.copy()
                            new_space[2]=frames/float(fps)
                            resampled_data=resample_volume(Origin=os.path.join(roots,files[j]),output='',new_spacing=new_space)
                            if sitk.GetArrayFromImage(resampled_data).shape[0]!=25:
                                print(f'resample failure: {roots} {files[j]} {frames}/{sitk.GetArrayFromImage(resampled_data).shape[0]} {spacing}/{new_space}')
                            else:
                                sitk.WriteImage(resampled_data,os.path.join(roots,files[j]))
                if np.std(frame_list)!=0:
                    print(f'frame_error {roots} {frame_list}')
fix_fps(nifti_path='/Users/airskcer/Downloads/NP_nifti-20230109/')
# fix_fps(nifti_path='/Users/airskcer/Downloads/RCM_nifti-20230107/')
# all_info=[]
# pats=os.listdir('E:/VST_fusion/')
# for i in range(len(pats)):
#     folder=os.path.join('E:/VST_fusion/',pats[i])
#     if os.path.isdir(folder):
#         print(folder)
#         fix_fps(nifti_path=folder,fix=True)

/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_mid.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_down.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_19.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_21.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_up.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_18.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_22.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11022043_HUANG_ZONG_LUN slice_20.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11037973_PEI_MENG_RAN slice_13.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230109/SAX_data/11037973_PEI_MENG_RAN slice_mid.nii.gz 16
/Users/airskcer/Downloads/NP_nifti-20230

In [ ]:
import random

def gen_nnunet_samples():
    root='E:/nifti_original_space/'
    batch=os.listdir(root)
    num=15
    mods=['SAX_data','4CH_data','4CH_LGE_data','SAX_LGE_data']
    exclude=[]
    error=[]
    for index in range(3):
        output=f'D:/samples/sample_F{index+1}/'
        for j in range(len(mods)):
            count=0
            for i in range(len(batch)):
                try:
                    batch_path=os.path.join(root,batch[i],mods[j])
                    try:
                        os.makedirs(batch_path.replace(root,output))
                    except:
                        pass
                    pats=os.listdir(batch_path)
                    random.shuffle(pats)
                    for k in range(min(len(pats),num)):
                        pat_folder=os.path.join(batch_path,pats[k])
                        target_folder=pat_folder.replace(root,output)
                        if pat_folder not in exclude and count<200:
                            shutil.copytree(pat_folder,target_folder)
                            count+=1
                            print(f"{pat_folder} -> {target_folder}")
                            exclude.append(pat_folder)
                except:
                    pass
    return exclude
# gen_nnunet_samples()

In [ ]:
# root='D:/samples/sample_B1/4CH_data/'
# pats=os.listdir(root)
# for i in range(len(pats)):
#     folder=pats[i].replace('.nii.gz','')
#     try:
#         os.makedirs(os.path.join(root,folder))
#     except:
#         pass
#     shutil.move(os.path.join(root,pats[i]),os.path.join(root,folder))

In [ ]:
# pd.DataFrame(all_info).to_csv('./fuwai_dataset/all_spacing_info.csv',index=False)
# data=sitk.ReadImage('e:/BaiduNetdiskDownload/DCM_nifti/4CH_data/11022931_CHU_GUI_ZHEN/11022931_CHU_GUI_ZHEN.nii.gz')
# data.GetSpacing()

In [ ]:
def change_resolution(dataroot='./test2/HCM_nfiti/'):
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            for i in range(len(files)):
                if files[i].endswith('.nii.gz'):
                    path=roots.replace('HCM_nfiti','HCM_me')
                    if not os.path.exists(path):
                        os.makedirs(path)
                    resample_volume(Origin=os.path.join(roots,files[i]),new_spacing=[1.8269231319427, 1.8269231319427,1],output=os.path.join(path,files[i]))
# change_resolution()

## Print all meteadata of a single dicom file or all dicom files in a folder

In [ ]:
def dicom_metadata(path):
    if os.path.isdir(path):
        for roots,dirs,files in os.walk(path):
            if len(dirs)==0:
                for i in range(len(files)):
                    if files[i].endswith('.dcm'):
                        info=pydicom.dcmread(os.path.join(roots,files[i]))
                        print(info)
    elif path.endswith('.dcm'):
        info=pydicom.dcmread(path)
        print(info)
    return info
# dicom_metadata('./New_dicom/11892346_ZHANG_YING/20190528/Cine FIESTA SA/')

In [ ]:
dicom_metadata('F:/resampled_dicom/ARVC_1_Resampled_dicom/11097334_FENG_ZHEN/20110117/4CH_LGE/')